### Separating Concerts from Programme

In [ ]:
import json
with open('nested_data.json') as f:
  data1 = json.load(f)

In [ ]:
print(len(data1)) # data1 has only one key value pair
print(len(data1['programs'])) # the value with the key "programs" is a list with length 13954
type(data1['programs'])

1
13954


list

In [ ]:
import pandas as pd
df = pd.json_normalize(data1["programs"])

In [ ]:
print(df.shape)
df.head()

(13954, 6)


,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37
3,1842-43,New York Philharmonic,"[{'Date': '1843-04-22T05:00:00Z', 'eventType':...",5887,"[{'workTitle': 'SYMPHONY NO. 2 IN D MAJOR, OP....",34ec2c2b-3297-4716-9831-b538310462b7
4,1843-44,New York Philharmonic,"[{'Date': '1843-11-18T05:00:00Z', 'eventType':...",305,"[{'workTitle': 'SYMPHONY NO. 7 IN A MAJOR, OP....",610a4acc-94e4-4cd6-bdc1-8ad020edc7e9


- following code shows that "programID" column have duplicates

In [ ]:
print(df["programID"].nunique())
print(df["programID"].isnull().sum())
print(df["programID"].dtype)
df["programID"].value_counts()[:3]

13932
0
object


,count
programID,
4917,2
12707,2
13972,2


- so, we choose "id" column which has no duplicates, as unique identifier.

In [ ]:
print(df["id"].nunique())
print(df["id"].isnull().sum())
print(df["id"].dtype)
df["id"].value_counts()[:3]

13954
0
object


,count
id,
38e072a7-8fc9-4f9a-8eac-3957905c0002,1
0080978c-2fbe-4c1c-8974-cdcda7b0b3e1,1
c24746f9-457b-48cb-a4df-6f65d7a60979,1


In [ ]:
df.concerts[0]

[{'Date': '1842-12-07T05:00:00Z',
  'eventType': 'Subscription Season',
  'Venue': 'Apollo Rooms',
  'Location': 'Manhattan, NY',
  'Time': '8:00PM'}]

In [ ]:
concert0_df = pd.json_normalize(df.concerts[0])
concert0_df

,Date,eventType,Venue,Location,Time
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM


In [ ]:
flag = False
rows_with_more_then_one_concert = []
for i in range(df.shape[0]):
  if len(pd.json_normalize(df.concerts[i])) > 1:
    flag = True
    rows_with_more_then_one_concert.append(i)

print(flag)
len(rows_with_more_then_one_concert)

True


4245

In [ ]:
df.iloc[rows_with_more_then_one_concert[0]]

,184
season,1878-79
orchestra,New York Symphony
concerts,"[{'Date': '1878-11-07T05:00:00Z', 'eventType':..."
programID,8764
works,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP...."
id,1254a8ad-0176-4e7e-8418-aedbd8492961


In [ ]:
concert1 = df.iloc[rows_with_more_then_one_concert[0]]['concerts']
concert1

[{'Date': '1878-11-07T05:00:00Z',
  'eventType': 'Subscription Season',
  'Venue': 'Steinway Hall',
  'Location': 'Manhattan, NY',
  'Time': '2:00PM'},
 {'Date': '1878-11-09T05:00:00Z',
  'eventType': 'Subscription Season',
  'Venue': 'Steinway Hall',
  'Location': 'Manhattan, NY',
  'Time': '8:00PM'}]

In [ ]:
concert1_df = pd.json_normalize(concert1)
concert1_df

,Date,eventType,Venue,Location,Time
0,1878-11-07T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",2:00PM
1,1878-11-09T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",8:00PM


In [ ]:
pd.concat([concert0_df, concert1_df], ignore_index=True)

,Date,eventType,Venue,Location,Time
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM
1,1878-11-07T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",2:00PM
2,1878-11-09T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",8:00PM


In [ ]:
print(df.iloc[0]['id'])
print(df.concerts[0])
df.iloc[0]

38e072a7-8fc9-4f9a-8eac-3957905c0002
[{'Date': '1842-12-07T05:00:00Z', 'eventType': 'Subscription Season', 'Venue': 'Apollo Rooms', 'Location': 'Manhattan, NY', 'Time': '8:00PM'}]


,0
season,1842-43
orchestra,New York Philharmonic
concerts,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."
programID,3853
works,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP...."
id,38e072a7-8fc9-4f9a-8eac-3957905c0002


In [ ]:
temp = rows_with_more_then_one_concert[0]
temp

184

In [ ]:
df.iloc[temp]['id']

'1254a8ad-0176-4e7e-8418-aedbd8492961'

In [ ]:
print(df.iloc[temp]['id'])
print(df.concerts[temp])
df.iloc[temp]

1254a8ad-0176-4e7e-8418-aedbd8492961
[{'Date': '1878-11-07T05:00:00Z', 'eventType': 'Subscription Season', 'Venue': 'Steinway Hall', 'Location': 'Manhattan, NY', 'Time': '2:00PM'}, {'Date': '1878-11-09T05:00:00Z', 'eventType': 'Subscription Season', 'Venue': 'Steinway Hall', 'Location': 'Manhattan, NY', 'Time': '8:00PM'}]


,184
season,1878-79
orchestra,New York Symphony
concerts,"[{'Date': '1878-11-07T05:00:00Z', 'eventType':..."
programID,8764
works,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP...."
id,1254a8ad-0176-4e7e-8418-aedbd8492961


In [ ]:
concert0_df["id"] = df.iloc[0]['id']
concert1_df["id"] = df.iloc[rows_with_more_then_one_concert[0]]['id']

# Concatenate the DataFrames
pd.concat([concert0_df, concert1_df], ignore_index=True)

,Date,eventType,Venue,Location,Time,id
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1878-11-07T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",2:00PM,1254a8ad-0176-4e7e-8418-aedbd8492961
2,1878-11-09T05:00:00Z,Subscription Season,Steinway Hall,"Manhattan, NY",8:00PM,1254a8ad-0176-4e7e-8418-aedbd8492961


In [ ]:
# Traverse through entire dataframe df and convert concerts into dataframe, add a column for "id" and then concat them togather
concerts_df = pd.json_normalize(df.iloc[0]['concerts'])
concerts_df['id'] = df.iloc[0]['id']

for i in range(1, df.shape[0]):
  temp_df = pd.json_normalize(df.iloc[i]['concerts'])
  temp_df['concert_id'] = df.iloc[i]['id']
  concerts_df = pd.concat([concerts_df, temp_df], ignore_index=True)

In [ ]:
print(concerts_df.shape)
concerts_df.head()

(21607, 7)


,Date,eventType,Venue,Location,Time,id,concert_id
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN
1,1843-02-18T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,NaN,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1843-04-07T05:00:00Z,Special,Apollo Rooms,"Manhattan, NY",8:00PM,NaN,894e1a52-1ae5-4fa7-aec0-b99997555a37
3,1843-04-22T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,NaN,34ec2c2b-3297-4716-9831-b538310462b7
4,1843-11-18T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",None,NaN,610a4acc-94e4-4cd6-bdc1-8ad020edc7e9


In [ ]:
pd.json_normalize(df.works[0])

,workTitle,conductorName,ID,soloists,composerName,movement,interval
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",NaN,NaN
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",NaN,NaN
3,NaN,NaN,0*,[],NaN,NaN,Intermission
4,OBERON,"Etienne, Denis G.",8834*3,[],"Weber, Carl Maria Von",Overture,NaN
5,ARMIDA,"Timm, Henry C.",8835*1,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Rossini, Gioachino",Duet,NaN
6,"FIDELIO, OP. 72","Timm, Henry C.",8837*6,"[{'soloistName': 'Horn, Charles Edward', 'solo...","Beethoven, Ludwig van","""In Des Lebens Fruhlingstagen...O spur ich nic...",NaN
7,"ABDUCTION FROM THE SERAGLIO,THE, K.384","Timm, Henry C.",8336*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Mozart, Wolfgang Amadeus","""Ach Ich liebte,"" Konstanze (aria)",NaN
8,"OVERTURE NO. 1, D MINOR, OP. 38","Timm, Henry C.",5543*,[],"Kalliwoda, Johann W.",NaN,NaN


In [ ]:
pd.json_normalize(df.works[0])['soloists'][2]

[{'soloistName': 'Scharfenberg, William',
  'soloistRoles': 'A',
  'soloistInstrument': 'Piano'},
 {'soloistName': 'Hill, Ureli Corelli',
  'soloistRoles': 'A',
  'soloistInstrument': 'Violin'},
 {'soloistName': 'Derwort, G. H.',
  'soloistRoles': 'A',
  'soloistInstrument': 'Viola'},
 {'soloistName': 'Boucher, Alfred',
  'soloistRoles': 'A',
  'soloistInstrument': 'Cello'},
 {'soloistName': 'Rosier, F. W.',
  'soloistRoles': 'A',
  'soloistInstrument': 'Contrabass'}]

In [ ]:
pd.json_normalize(pd.json_normalize(df.works[0])['soloists'][2])

,soloistName,soloistRoles,soloistInstrument
0,"Scharfenberg, William",A,Piano
1,"Hill, Ureli Corelli",A,Violin
2,"Derwort, G. H.",A,Viola
3,"Boucher, Alfred",A,Cello
4,"Rosier, F. W.",A,Contrabass


### Creating a dataframe for df['works']

In [ ]:
print(type(df.works[0]))
df.works[0]

<class 'list'>


[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP.67',
  'conductorName': 'Hill, Ureli Corelli',
  'ID': '52446*',
  'soloists': [],
  'composerName': 'Beethoven,  Ludwig  van'},
 {'workTitle': 'OBERON',
  'composerName': 'Weber,  Carl  Maria Von',
  'conductorName': 'Timm, Henry C.',
  'ID': '8834*4',
  'soloists': [{'soloistName': 'Otto, Antoinette',
    'soloistRoles': 'S',
    'soloistInstrument': 'Soprano'}],
  'movement': '"Ozean, du Ungeheuer" (Ocean, thou mighty monster), Reiza (Scene and Aria), Act II'},
 {'workTitle': 'QUINTET, PIANO, D MINOR, OP. 74',
  'ID': '3642*',
  'soloists': [{'soloistName': 'Scharfenberg, William',
    'soloistRoles': 'A',
    'soloistInstrument': 'Piano'},
   {'soloistName': 'Hill, Ureli Corelli',
    'soloistRoles': 'A',
    'soloistInstrument': 'Violin'},
   {'soloistName': 'Derwort, G. H.',
    'soloistRoles': 'A',
    'soloistInstrument': 'Viola'},
   {'soloistName': 'Boucher, Alfred',
    'soloistRoles': 'A',
    'soloistInstrument': 'Cello'},
   {

In [ ]:
works_list = []
for i in range(df.shape[0]):
  temp_list = df.works[i]
  for work in temp_list:
    work['concert_id'] = df.id[i]

  works_list.extend(temp_list)

In [ ]:
works_df = pd.json_normalize(works_list)
works_df.head()

,workTitle,conductorName,ID,soloists,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,NaN,NaN,NaN,NaN,NaN
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von",38e072a7-8fc9-4f9a-8eac-3957905c0002,"""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0*,[],NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,Intermission,NaN,NaN,NaN,NaN
4,OBERON,"Etienne, Denis G.",8834*3,[],"Weber, Carl Maria Von",38e072a7-8fc9-4f9a-8eac-3957905c0002,Overture,NaN,NaN,NaN,NaN,NaN


### separating soloist from works_df

In [ ]:
works_df.head()

,workTitle,conductorName,ID,soloists,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,NaN,NaN,NaN,NaN,NaN
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von",38e072a7-8fc9-4f9a-8eac-3957905c0002,"""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0*,[],NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,NaN,Intermission,NaN,NaN,NaN,NaN
4,OBERON,"Etienne, Denis G.",8834*3,[],"Weber, Carl Maria Von",38e072a7-8fc9-4f9a-8eac-3957905c0002,Overture,NaN,NaN,NaN,NaN,NaN


In [ ]:
works_df.describe()

,workTitle,conductorName,ID,soloists,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._
count,71422,66314,82793,82793,71427,82793,24989,11366,227,114,5,5
unique,11269,794,16648,8295,2766,13907,4648,4,84,37,4,4
top,"MEISTERSINGER, DIE, WWV 96","Damrosch, Walter",0*,[],"Wagner, Richard",f549e93f-b35f-4824-b0d5-d543953535f8,Overture,Intermission,The Swan of Tuonela,(name of Marmein dance),FIFTHS,"QUARTET, STRING, OP. 76, NO. 2, D MINOR, H.III..."
freq,784,5427,11278,56725,4929,50,3419,11278,80,18,2,2


In [ ]:
works_df1 = works_df[ works_df['soloists'].apply(lambda x: len(x)) != 0 ]
works_df1.shape

(26068, 12)

In [ ]:
works_df1.describe()

,workTitle,conductorName,ID,soloists,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._
count,26064,21176,26068,26068,26068,26068,8461,0,128,84,4,4
unique,7325,627,10318,8294,1946,10422,3001,0,67,25,3,3
top,PORGY AND BESS,"Damrosch, Walter",9004*14,"[{'soloistName': 'No Soloist', 'soloistRoles':...","Beethoven, Ludwig van",f549e93f-b35f-4824-b0d5-d543953535f8,Prelude and Liebestod (with or without soloist),NaN,The Swan of Tuonela,(name of Marmein dance),FIFTHS,"QUARTET, STRING, OP. 76, NO. 2, D MINOR, H.III..."
freq,387,1670,294,248,1347,50,294,NaN,27,16,2,2


In [ ]:
works_df1['new_column'] = works_df1[['ID', 'composerName', 'concert_id']].astype(str).agg('_'.join, axis=1)

<ipython-input-31-6948983e53d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  works_df1['new_column'] = works_df1[['ID', 'composerName', 'concert_id']].astype(str).agg('_'.join, axis=1)


In [ ]:
works_df1.describe()

,workTitle,conductorName,ID,soloists,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._,new_column
count,26064,21176,26068,26068,26068,26068,8461,0,128,84,4,4,26068
unique,7325,627,10318,8294,1946,10422,3001,0,67,25,3,3,25994
top,PORGY AND BESS,"Damrosch, Walter",9004*14,"[{'soloistName': 'No Soloist', 'soloistRoles':...","Beethoven, Ludwig van",f549e93f-b35f-4824-b0d5-d543953535f8,Prelude and Liebestod (with or without soloist),NaN,The Swan of Tuonela,(name of Marmein dance),FIFTHS,"QUARTET, STRING, OP. 76, NO. 2, D MINOR, H.III...","5970*_Unspecified,_83d202fe-e59b-4c9b-92e4-de5..."
freq,387,1670,294,248,1347,50,294,NaN,27,16,2,2,9


In [ ]:
soloists_list = []
for i in range(works_df1.shape[0]):
  temp_soloists_list = works_df1.iloc[i]['soloists']
  for soloist in temp_soloists_list:
    soloist['concert_id'] = works_df1.iloc[i]['concert_id']
    soloist['works_ID'] = works_df1.iloc[i]['ID']
    soloist['works_composerName'] = works_df1.iloc[i]['composerName']

  soloists_list.extend(temp_soloists_list)

In [ ]:
soloists_df = pd.json_normalize(soloists_list)
soloists_df.head()

,soloistName,soloistRoles,soloistInstrument,concert_id,works_ID,works_composerName
0,"Otto, Antoinette",S,Soprano,38e072a7-8fc9-4f9a-8eac-3957905c0002,8834*4,"Weber, Carl Maria Von"
1,"Scharfenberg, William",A,Piano,38e072a7-8fc9-4f9a-8eac-3957905c0002,3642*,"Hummel, Johann"
2,"Hill, Ureli Corelli",A,Violin,38e072a7-8fc9-4f9a-8eac-3957905c0002,3642*,"Hummel, Johann"
3,"Derwort, G. H.",A,Viola,38e072a7-8fc9-4f9a-8eac-3957905c0002,3642*,"Hummel, Johann"
4,"Boucher, Alfred",A,Cello,38e072a7-8fc9-4f9a-8eac-3957905c0002,3642*,"Hummel, Johann"


In [ ]:
print(soloists_df.shape)
soloists_df.drop_duplicates(inplace = True)
print(soloists_df.shape)

(56194, 6)
(55928, 6)


In [ ]:
works_df.drop('soloists', axis=1, inplace=True)

In [ ]:
print(works_df.shape)
works_df.drop_duplicates(inplace = True)
print(works_df.shape)

(82793, 11)
(82571, 11)


In [ ]:
df.drop(['concerts', 'works'], axis=1, inplace=True)

In [ ]:
# Print the shape of the DataFrame 'df' before dropping duplicates
print("Before dropping duplicates:", df.shape)

# Drop duplicate rows inplace
df.drop_duplicates(inplace=True)

# Print the shape of the DataFrame 'df' after dropping duplicates
print("After dropping duplicates:", df.shape)


Before dropping duplicates: (13954, 4)
After dropping duplicates: (13954, 4)


- The following code will show that copying the columns 'ID', 'composerName', 'concert_id' from works_df to soloists_df was right.
- We can use a combination of these columns as a foreign key of table soloists_df which connects it with works_df.

In [ ]:
works_df['new_column'] = works_df[['ID', 'composerName', 'concert_id']].astype(str).agg('_'.join, axis=1)
works_df.describe()

,workTitle,conductorName,ID,composerName,concert_id,movement,interval,movement.em,movement._,workTitle.em,workTitle._,new_column
count,71286,66190,82571,71291,82571,24938,11280,227,114,5,5,82571
unique,11269,794,16648,2766,13907,4648,4,84,37,4,4,82571
top,"MEISTERSINGER, DIE, WWV 96","Damrosch, Walter",0*,"Wagner, Richard",f549e93f-b35f-4824-b0d5-d543953535f8,Overture,Intermission,The Swan of Tuonela,(name of Marmein dance),FIFTHS,"QUARTET, STRING, OP. 76, NO. 2, D MINOR, H.III...","52446*_Beethoven, Ludwig van_38e072a7-8fc9-4..."
freq,784,5405,11192,4923,50,3418,11192,80,18,2,2,1


In [ ]:
# Saving works_df to a CSV file
works_df.to_csv('works_df.csv', index=False)

# Saving soloists_df to a CSV file
soloists_df.to_csv('soloists_df.csv', index=False)

# Saving df to a CSV file
df.to_csv('programs_df.csv', index=False)
